This script prepares the datasets for the situation testing pipeline. All datasets are stored within the 'data' folder. Original datasets are read from the 'data/raw' folder; modified datasets are saved into the 'data' folder with 'clean_' as a prefix. Here, we also indicate the origin of all original datasets.

In [1]:
import os
import pandas as pd
import numpy as np
# ...

In [2]:
# set working directory - note: all code runs from the src folder
wrk_dir = os.path.dirname(os.getcwd())
print(wrk_dir)
# set data path
data_path = wrk_dir + '\\' + 'data' + '\\'
print(data_path)

C:\Users\Jose Alvarez\Documents\Projects\CounterfactualSituationTesting
C:\Users\Jose Alvarez\Documents\Projects\CounterfactualSituationTesting\data\


### German Credit Data (Revised)

### Law School Data

The original dataset was taken from [Kusner et al. (2017)](https://github.com/mkusner/counterfactual-fairness). This dataset in turn comes from a survey study conducted in the late 1980s by the Law School Admission Council. For the original study see [Wightman (1998)](https://archive.lawschooltransparency.com/reform/projects/investigations/2015/documents/NLBPS.pdf).

Notes: (1) values for the var 'sex': sex==1 for women, sex==2 for men; (2) nowadays the LSAT ranges from 120 - 180, but from 1981 to 1991 it used to be a 48-point scale, ranging from 10 to 48 as in the var 'LSAT'; (3) the data doesn't contain a decision outcome. 

In [3]:
data2 = pd.read_csv(data_path + 'raw\\' + 'law_data.csv', index_col=False, )
print(data2.shape)
print(data2.columns.tolist())
data2.head(5)

(21791, 9)
['Unnamed: 0', 'race', 'sex', 'LSAT', 'UGPA', 'region_first', 'ZFYA', 'sander_index', 'first_pf']


,Unnamed: 0,race,sex,LSAT,UGPA,region_first,ZFYA,sander_index,first_pf
0,0,White,1,39.0,3.1,GL,-0.98,0.782738,1.0
1,1,White,1,36.0,3.0,GL,0.09,0.735714,1.0
2,2,White,2,30.0,3.1,MS,-0.35,0.670238,1.0
3,5,Hispanic,2,39.0,2.2,NE,0.58,0.697024,1.0
4,6,White,1,37.0,3.4,GL,-1.26,0.786310,1.0


In [4]:
# drop 'PO' like in Kusner et al.
data2 = data2[data2['region_first']!='PO']
data2.reset_index(drop=True, inplace=True)

In [5]:
# map sex
data2 = data2.drop(columns=['Unnamed: 0', 'sander_index', 'first_pf', 'region_first'])
data2['sex'] = data2['sex'].map({1: 'Female', 2: 'Male'})

In [6]:
data2.groupby('sex').count()

,race,LSAT,UGPA,ZFYA
sex,,,,
Female,9537,9537,9537,9537
Male,12253,12253,12253,12253


In [7]:
# map race
data2.groupby('race').count()

,sex,LSAT,UGPA,ZFYA
race,,,,
Amerindian,99,99,99,99
Asian,845,845,845,845
Black,1282,1282,1282,1282
Hispanic,488,488,488,488
Mexican,389,389,389,389
Other,293,293,293,293
Puertorican,110,110,110,110
White,18284,18284,18284,18284


In [8]:
# two 'simpler' options

def r_nonwhite(x):
    if x=='White':
        return 'White'
    else:
        return 'NonWhite'

def r_simpler(x):
    if x=='Hispanic' or x=='Mexican' or x=='Puertorican':
        return 'Latino'
    if x=='White':
        return 'White'
    if x=='Amerindian' or x=='Other':
        return 'Other'
    if x=='Black':
        return 'Black'
    if x=='Asian':
        return 'Asian'


In [9]:
data2['race_nonwhite'] = data2['race'].map(r_nonwhite)
data2['race_simpler']  = data2['race'].map(r_simpler)

In [10]:
data2.groupby('race_nonwhite').count()

,race,sex,LSAT,UGPA,ZFYA,race_simpler
race_nonwhite,,,,,,
NonWhite,3506,3506,3506,3506,3506,3506
White,18284,18284,18284,18284,18284,18284


In [11]:
data2.groupby('race_simpler').count()

,race,sex,LSAT,UGPA,ZFYA,race_nonwhite
race_simpler,,,,,,
Asian,845,845,845,845,845,845
Black,1282,1282,1282,1282,1282,1282
Latino,987,987,987,987,987,987
Other,392,392,392,392,392,392
White,18284,18284,18284,18284,18284,18284


In [12]:
# store in data folder
data2.to_csv(data_path + '\\' + 'clean_LawData.csv', sep='|', index=False)